# `Tag-Tag` graphs analysis using `memgraph`

This is just a try

In [ ]:
from gqlalchemy import Memgraph

memgraph = Memgraph("127.0.0.1", 7687)

## First, create the first graph, let's call it `COMMON_PRO`

We would like to do projections, how? Two projections:

- Project the bipartite graph `Project-Tag` on the `Tag`
- Project the bipartite graph `Lender-Tag` on the `Tag`

The weight would be IoU

In [ ]:
results = memgraph.execute_and_fetch(
    """
MATCH (t1:Tag)<-[:TAGGED_WITH]-(p:Project)-[:TAGGED_WITH]->(t2:Tag)
WHERE ID(t1) > ID(t2)
WITH t1, t2, COUNT(DISTINCT p) as intersection_count
CALL {WITH t1 MATCH (p1:Project)-[:TAGGED_WITH]->(t1) RETURN COUNT(DISTINCT p1) as t1pro}
CALL {WITH t2 MATCH (p2:Project)-[:TAGGED_WITH]->(t2) RETURN COUNT(DISTINCT p2) as t2pro}
WITH t1, t2, intersection_count, t1pro + t2pro - intersection_count as union_count, 
      intersection_count * 1.0 / (t1pro + t2pro - intersection_count) as theweight
CREATE (t1)-[i:COMMON_PRO {weight: theweight}]->(t2)
RETURN COUNT(i);
"""
)
list(results)

# TODO: Create `COMMON_LENDER` graph

To do this, first create the `INTEREST` graph, which is a `Lender-Tag` graph.

Then project it to `Tag`, but what is the weight?




# Analysis the `COMMON_PRO` relationship

View the relationship in the memgraph lab

```cypher
MATCH p=(:Tag)-[i:COMMON_PRO]-(:Tag)
RETURN p
```

## Centralarity

Calculate pagerank, then store the ranks in property `common_pro_rank` of each tag node

In [ ]:
results = memgraph.execute_and_fetch(
    """
MATCH p=(:Tag)-[i:COMMON_PRO]-(:Tag)
WITH project(p) AS tagtag
CALL pagerank.get(tagtag) YIELD node, rank
SET node.common_pro_rank = rank
RETURN node.name, rank
ORDER BY rank DESC
"""
)
list(results)

to visualize the pagerank, we should use the memgraph lab again

query:

```cypher
MATCH (t:Tag)
WITH t
ORDER BY t.rank DESC
LIMIT 2
MATCH path=(t)<-[:COMMON_LOANS]-(m:Tag)
RETURN path;
```

Add the following style to the *System Style*. It helps changing the node size based on the rank:

```text
@NodeStyle HasProperty(node, "common_pro_rank") {
  size: Round(Add(Mul(Property(node, "common_pro_rank"), 50), 2))
}
```

## Community finding

## `Louvain` community

In [ ]:
results = memgraph.execute_and_fetch(
    """
MATCH (t:Tag), (:Tag)-[i:COMMON_PRO]->(:Tag)
WITH collect(t) AS nodes,
     collect(i) AS relas
CALL community_detection.get_subgraph(nodes, relas, "weight")
YIELD node, community_id
SET node.common_pro_community_id = community_id
RETURN node.name, community_id
ORDER BY community_id;
"""
)
list(results)

In [ ]:
results = memgraph.execute_and_fetch(
    """
MATCH (t:Tag)
WITH DISTINCT t.community_id as community_id
RETURN count(community_id ) as num_of_clusters;
    """
)
# you will get only 1 result
result = list(results)[0]

# don't forget that results are saved in a dict
print(f"Number of clusters: {result['num_of_clusters']}")

display Tags with community

```
MATCH p=(Tag)-[:COMMON_LOANS]->(:Tag)
RETURN p
```

style change


```
// Cache map to keep a selected color for each community
Define(ColorByInt, AsMap())
Define(GetColorByInt, Function(i, Coalesce(
  Get(ColorByInt, AsText(i)),
  Set(ColorByInt, AsText(i), GetNextColor())
)))

// Feel free to uncomment the lines below to set up a custom style for the specific node property
@NodeStyle HasProperty(node, "community_id") {
  Define(COLOR, GetColorByInt(Property(node, "community_id")))

  color: COLOR
  color-hover: Lighter(COLOR)
  color-selected: Darker(COLOR)
}
```